# Baxter Vision Introduction

Learn how make baxter use its cameras and distance sensors.

##### ENGG 4460 University of Guelph, Patrick Wspanialy

Import python modules needed to control Baxter

In [1]:
import pprint
import math
import rospy
import baxter_interface
from baxter_interface import RobotEnable
from baxter_interface import CameraController
from sensor_msgs.msg import Image, Range
import cv2
import cv_bridge
from cv_bridge import CvBridge, CvBridgeError
bridge = CvBridge()
pp = pprint.PrettyPrinter(indent=4, width=1)

Initialize a new ROS node and name it "camera_example" 

In [ ]:
rospy.init_node('camera_example')

Create variables to access Baxter's two hand cameras.

In [ ]:
right_hand_camera = CameraController('right_hand_camera')
left_hand_camera = CameraController('left_hand_camera')

Set some of the properties of the right hand camera. Once the properties are set, we need to "open" the camera to activate it. 

In [ ]:
right_hand_camera.resolution = (960, 600)
right_hand_camera.gain = 0
right_hand_camera.open()

To helps us understand how far away an object is from the hand, we can use Baxter's range sensors. They have a maximum range of ~0.3m. If you see ~65, that means it's not seeing anything in range.

We define a class that will take care of reading the range sensors for us.

In [ ]:
class BaxterRangeSensor(object):
    def __init__(self):
        self.distance = {}
        root_name = "/robot/range/"
        sensor_name = ["left_hand_range/state","right_hand_range/state"]
        self.__left_sensor = rospy.Subscriber(root_name + sensor_name[0], Range, callback=self.__sensorCallback, callback_args="left", queue_size=1)
        self.__right_sensor = rospy.Subscriber(root_name + sensor_name[1], Range, callback=self.__sensorCallback, callback_args="right", queue_size=1)
       
    def __sensorCallback(self,msg,side):
       self.distance[side] = msg.range

range_sensors = BaxterRangeSensor()

Now we can access the distances seen by the range sensors.

In [ ]:
pp.pprint(range_sensors.distance)

Now let's take a video with our right hand camera, add some graphics to it, and show it on Baxter's face.

In [ ]:
class BaxterRightHandVision(object):
    def __init__(self):
        right_hand_image_topic = '/cameras/right_hand_camera/image'
        right_hand_image_subscriber = rospy.Subscriber(right_hand_image_topic, Image, callback=self.__processImageCallback, queue_size=1)
        display_topic = '/robot/xdisplay'
        self.__display_publisher = rospy.Publisher(display_topic, Image, queue_size=1)
        
    def __processImageCallback(self, message):
        # convert ROS iamge to OpenCV image
        cv2_image = bridge.imgmsg_to_cv2(message)
        
        # this is where we do our image processing. 
        # We're just going to draw a red rectangle and the distance we previously got from our distance sensor
        # note, that in OpenCV, colors are represented as (blue, green, red), rather than RGB
        cv2.rectangle(
                      cv2_image,
                      pt1=(280,200),
                      pt2=(680,400),
                      color=(0,0,255),
                      thickness=5
                      )
        cv2.putText(
                    cv2_image,
                    text='%.2f' % range_sensors.distance['right'],
                    org=(400,500),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=2,
                    color=(255,255,0)
                    )

        # convert OpenCV image back to ROS image
        ros_image = bridge.cv2_to_imgmsg(cv2_image)
        
        # publish our modified image to the display
        self.__display_publisher.publish(ros_image)

    def __findToy(self, img):
        try:
            detection = detect_toy(example_image)
        except (TimeoutError, ):
            print("Detection has failed, un")
        
right_hand_vision = BaxterRightHandVision()